In [ ]:
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np

#create the list of dictinary formatted strings by extracting data item
#from twitter API scrape
#files = ['twtgun3.24.1.txt', 'twtgun3.24.2.txt','twtgun3.24.3.txt', 'twtgun3.24.4.txt']

group = []
for i in range(1,11):
   
    with open('twtgun3.24.' + str(i) + '.txt') as fhand:
        for item in fhand:
            dic = {}
            x = re.findall("^{'created_at': \'(.*?)\'", item)
            if len(x) > 0:
                dic["created_at"] = str(x[0])
            else:
                dic["created_at"] = None
            y = re.findall("{'full_text': \'(.*?)\'", item)
            if len(y) > 0:
                dic["full_text"] = str(y[0])
            else:
                dic["full_text"] = None
            z = re.findall(" 'text': \'(.*?)\'", item)
            if len(z) > 0:
                dic["text"] = str(z[0])
            else:
                dic["text"] = None
            a = re.findall("\'location\': \'(.*?)\'", item)
            if len(a) > 0:
                dic["location"] = str(a[0])
            else:
                dic["location"] = None
            b = re.findall("\'time_zone\': \'(.*?)\'", item)
            if len(b) > 0:
                dic["time_zone"] = str(b[0])
            else:
                dic["time_zone"] = None
            group.append(dic)

    fhand.close()

#convert the list into json object

with open('data.json', 'w') as fp:
    json.dump(group, fp)

    # convert json file into pandas DataFrame
df = pd.read_json('data.json')

In [ ]:
df.sample(5)

In [ ]:
df2 = df.reindex(columns = ['created_at','location','time_zone','text','full_text'])
pd.options.display.max_rows=10
#print(df2)

In [ ]:
df2['text'] = df2['text'].replace(r'\\n', '', regex=True).str.lower()
df2['text'] = df2['text'].str.strip()
df2['full_text'] = df2['full_text'].replace(r'\\n', '', regex=True).str.lower()
df2['full_text'] = df2['full_text'].str.strip()

In [ ]:
print(df2.head())

In [ ]:
#df2['full_text'].str.findall('\\n')

In [ ]:
#make copy of df2 and name it df3
df3 = df2

In [ ]:
#Take 'full_text' when available, otherwise take 'text'
df3['tweet'] = df3['full_text'].where(pd.notna(df3['full_text']), other = df3['text'])
#Use time_zone for proxy of location when location data is missing
df3['location_'] = df3['location'].where(pd.notna(df3['location']), other = df3['time_zone'])

In [ ]:
pd.isna(df3['tweet']).sum()

In [ ]:
#drop columns 'text' & 'full_text'
df3 = df3.drop(['text', 'full_text', 'location', 'time_zone'], axis=1)
df3.rename(columns = {'location_':'location'}, inplace = True)
df3.reindex(columns = ['created_at','location','tweet'])

#drop rows all elements are NaN

df3 = df3.dropna(how='all')

In [ ]:
#rename locations taken from time_zone ex) Eastern Time (US & Canada) -> US & Canada EastCoast etc 
#df3.to_csv('test2.csv')

In [ ]:
#Etract hashtag from 'tweet' and create a new cloumn 'hashtag'
df3['hashtag'] = df3['tweet'].str.findall(r'#.*?\s')

In [ ]:
df3 = df3.reindex(columns = ['created_at','location','tweet','hashtag'])
print(df3.sample(5))

In [ ]:
df3['created_at'] = df3['created_at'].apply(lambda x: x.round('min'))

In [ ]:
df3.groupby('created_at').count()

In [ ]:
df3.groupby('created_at').count()['tweet']

In [ ]:
df3.groupby('created_at').count()['tweet'].plot()

In [ ]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

In [ ]:
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [ ]:
tweet = df3['tweet'][10]

In [ ]:
print(preprocess(tweet))

In [ ]:
df4 = df3.dropna(axis=0, inplace=False)
df4.head()

In [ ]:
#df4['test'] = df4['tweet'].apply(lambda x: type(x))
#df4.to_csv('test3.csv')
df4.loc[:,'preprocess_tweet'] = df4.loc[:,'tweet'].apply(lambda x : preprocess(x))
df4.head()

In [ ]:
#nltk.download('stopwords')
from nltk.corpus import stopwords
import string

punc = list(string.punctuation)
stop = stopwords.words('english') + punc + ['rt', 'via', '’', 'amp']

In [ ]:
terms_nostop = [term for term in preprocess(tweet) if term not in stop]

In [ ]:
# There are 37487 rows in df4. for each item (list) in the 'preprocess_tweet' column, check if it's non-empty, them 
# remove stop words from the item and append it to the longer 'items' list. Since the data is large, will run the loop
# for 4 times (10,000 x 3 + 7487 X 1)
# increase data rate by jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

items = []
for item in df4['preprocess_tweet']:
    if len(item) !=0:
        for i in item:
            if i not in stop and not i.startswith(('#', '@')):
                items.append(i.strip())
            else:
                continue  
    else:
        continue
        
print(items[:10])

In [ ]:
import operator
from collections import Counter

count_all = Counter()
count_all.update(items)
print(count_all.most_common(30))    

In [ ]:
hashlist = []
for item in df4['hashtag']:
    if len(item) !=0:
        for i in item:
            if i.startswith('#'):
                hashlist.append(i.strip())
            else:
                continue  
    else:
        continue
        
print(hashlist[:10])

In [ ]:
count_hash = Counter()
count_hash.update(hashlist)
print(count_hash.most_common(30))

In [ ]:
# to remove emojis from tokens

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F500"  # symbols & pictographs
        u"\U0001F520-\U0001F52F"
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)                          
        u"\U0001F910-\U0001F96B" 
        u"\U0001F52B"
        u"\U0001F5E3"             # speech
        u"\U0001F5F3"             # vote 
        u"\U0001F91B-\U0001F939"                   
        u"\U0001F191-\U0001F19A"
        u"\U0001F595"
                             "]+", flags = re.UNICODE)

In [ ]:
items_noemoji = []
for item in items:
    i = emoji_pattern.sub(r'', item)
    items_noemoji.append(i)
    
print(items_noemoji[:10])

In [ ]:
from nltk import bigrams

terms_bigram = list(bigrams(items_noemoji))
count_bigram = Counter()
count_bigram.update(terms_bigram)
print(count_bigram.most_common(30))

In [ ]:
import vincent
vincent.core.initialize_notebook()

hash_freq = count_hash.most_common(30)  #create list of tuples
labels, freq = zip(*hash_freq)  #seperate the above into 1. tuple of labels & 2. tuple of counts
data = {'data': freq, 'x': labels} #create dictionary of tuples
bar = vincent.Bar(data, iter_idx='x')
bar.display()

In [ ]:
beach_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#hermosabeach' in df4['tweet'].iloc[i]:
        beach_hash_time.append(df4['created_at'].iloc[i])
        
print(beach_hash_time[:10])

In [ ]:
# Creating datetimeindex for time series data for pandas. Twitter streaming data is based on UTC time.
# From US & Canada ETS time, it's 4 hours earlier. Adjust for daytime savings.

from pandas.tseries.offsets import Hour
one_hour = Hour(1)

idx = pd.DatetimeIndex(beach_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(beach_hash_time)
#ones
beach_hash = pd.Series(ones, index=idx_est_ds)
per_minute = beach_hash.resample('1min').sum().fillna(0)

In [ ]:
time_chart = vincent.Line(per_minute)
time_chart.axis_titles(x='Time', y='Hashtag frequencies')
time_chart.display()

In [ ]:
march_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#marchforourlives' in df4['tweet'].iloc[i]:
        march_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(march_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(march_hash_time)
#ones
march_hash = pd.Series(ones, index=idx_est_ds)
march_per_minute = march_hash.resample('1min').sum().fillna(0)

In [ ]:
enough_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#enough' in df4['tweet'].iloc[i]:
        enough_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(enough_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(enough_hash_time)
#ones
enough_hash = pd.Series(ones, index=idx_est_ds)
enough_per_minute = enough_hash.resample('1min').sum().fillna(0)

In [ ]:
again_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#neveragain' in df4['tweet'].iloc[i]:
        again_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(again_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(again_hash_time)
#ones
again_hash = pd.Series(ones, index=idx_est_ds)
again_per_minute = again_hash.resample('1min').sum().fillna(0)

In [ ]:
control_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#guncontrol' in df4['tweet'].iloc[i]:
        control_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(control_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(control_hash_time)
#ones
control_hash = pd.Series(ones, index=idx_est_ds)
control_per_minute = control_hash.resample('1min').sum().fillna(0)

In [ ]:
violence_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#gunviolence' in df4['tweet'].iloc[i]:
        violence_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(violence_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(violence_hash_time)

violence_hash = pd.Series(ones, index=idx_est_ds)
violence_per_minute = violence_hash.resample('1min').sum().fillna(0)

In [ ]:
school_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#schoolshooting' in df4['tweet'].iloc[i]:
        school_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(school_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(school_hash_time)
#ones
school_hash = pd.Series(ones, index=idx_est_ds)
school_per_minute = school_hash.resample('1min').sum().fillna(0)

In [ ]:
time_chart = vincent.Line(school_per_minute)
time_chart.axis_titles(x='Time', y='Hashtag frequencies')
time_chart.display()

In [ ]:
match_data = dict(Beach=per_minute, March=march_per_minute, Enough=enough_per_minute, Again=again_per_minute, Control=control_per_minute, Violence=violence_per_minute)
all_matches = pd.DataFrame(data = match_data,index=march_per_minute.index)
all_matches

In [ ]:
time_chart_top3 = vincent.Line(all_matches[['March', 'Enough', 'Again']])
time_chart_top3.axis_titles(x='Time', y='Freq')
time_chart_top3.legend(title='Top 3 Most popular hashtags')
time_chart_top3.display()

In [ ]:
time_chart_next3 = vincent.Line(all_matches[['Control', 'Violence', 'Beach']])
time_chart_next3.axis_titles(x='Time', y='Freq')
time_chart_next3.legend(title='Top 4 to 6 Most popular hashtags')
time_chart_next3.display()

In [ ]:
pro_guncontrol = []  #collection of words which indictaes the tweet is pro guncontrol
anti_guncontrol = []  #collection of words which indicates the tweet is anti guncontrol

In [ ]:
df4.loc[:,'processed_location'] = df4.loc[:,'location'].apply(lambda x: emoji_pattern.sub(r'', x))
hashtag = re.compile(r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)')
number = re.compile(r'(?:(?:\d+,?)+(?:\.?\d+)?)')

df4.loc[:,'processed_location2'] = df4.loc[:,'processed_location'].apply(lambda x: hashtag.sub(r'', x))
df4.loc[:,'processed_location3'] = df4.loc[:, 'processed_location2'].apply(lambda x: number.sub(r'', x))
df4 = df4.drop(['processed_location2', 'processed_location'], axis=1)

In [ ]:
df4['clean_states'] = df4['processed_location3'].apply(lambda x: re.findall(r'[\w+],\s(\w+[\s|\.]*\w+)', x))

In [ ]:
df4['clean_states2'] = df4['processed_location3'].apply(lambda x : re.findall(r'(\w+[\s]*[\w+]+), USA', x))

In [ ]:
df4.to_csv('location.csv')